# Datatransformation

In [1]:
#kernel = python37
from math import sqrt
import pandas as pd
import numpy as np
import sklearn
from sklearn.neural_network import MLPRegressor
import pandas as pd
import matplotlib.pyplot as plt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from datetime import datetime
from keras.layers import Bidirectional
import datetime
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import tensorflow as tf


#********time check
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
print("Last update =", dt_string)

Last update = 2022/04/11 12:57:06


In [2]:
# first of all, determine the length of historical data you want to train on (n_steps_in) and forecast (n_steps_out)
# then append the sequence on a list, then append to a list, finally turn it into array (which can be further turned into dataframes)

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def series_to_supervised(data, window=1, lag=1, dropnan=True):
    #input data is required to be a dataframe
    data = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window-1, 0, -1): #because we want to import the t-23 on our first columns, the number need to be called reversely
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    
    
    # Target timestep (t=lag)
    for i in range(-1,-(lag+1),-1): #the import of t+1 ~ t+5 is in the foward direction, the shifting values should be negative
        cols.append(data.shift(i))
    for i in range(1,lag+1,1):
        names += [('%s(t+%d)' % (col, i)) for col in data.columns]

    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [3]:
def main():
    # 1. import train and test raw 2. select the date range for training (only train data is required) 3. resample the train and test data

    # choose the start and end date for training dataset
    start_date_train = '2021-12-23 00:00'
    end_date_train = '2022-1-15 23:59'

    start_date_test = '2022-01-16 00:00'
    end_date_test = '2022-01-22 23:59'

    for i in range(1, 9) : # There are seven datasets
        

        problem = i
        if problem ==1:
            traindata = pd.read_csv("../Exp-1/data/train/obs/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= "obs"

        if problem ==2:
            traindata = pd.read_csv("../Exp-1/data/train/sg5/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'sg5'

        if problem ==3:
            traindata = pd.read_csv("../Exp-1/data/train/sg7/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'sg7'

        if problem ==4:
            traindata = pd.read_csv("../Exp-1/data/train/sg9/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'sg9'

        if problem ==5:
            traindata = pd.read_csv("../Exp-1/data/train/ew2/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'ew2'

        if problem ==6:
            traindata = pd.read_csv("../Exp-1/data/train/ew3/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'ew3'

        if problem ==7:
            traindata = pd.read_csv('../Exp-1/data/train/ew4/train_dataset.csv',index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'ew4'

        if problem ==8:
            traindata = pd.read_csv("../Exp-1/data/train/or/train_dataset.csv",index_col = 0)
            traindata.index = pd.to_datetime(traindata.index,format = '%Y-%m-%d %H:%M' )
            traindata = traindata[start_date_train:end_date_train]
            name= 'or'

        #share the same test data
        testdata= pd.read_csv("../Exp-1/data/test/test_dataset.csv",index_col = 0)
        testdata.index = pd.to_datetime(testdata.index,format = '%Y-%m-%d %H:%M' )
        testdata = testdata[start_date_test:end_date_test]


        train = traindata.iloc[:,:]
        test = testdata.iloc[:,:]

        # choose a number of time steps
        n_steps_in, n_steps_out = 24, 3 #only use t-4 ~ t for training, and predict t+1 ~ t+10

        # split into samples
        #train_X, train_Y = split_sequence(train, n_steps_in, n_steps_out)
        train = series_to_supervised(train, n_steps_in, n_steps_out) #the output tyep is dataframe 
        train.reset_index(drop=True, inplace=True)
        #print(train)
        # train = np.concatenate((train_X, train_Y), axis=1)
        # train = pd.DataFrame(train, columns = ['Input1', 'Input2','Input3','Input4','Input5','Output1','Output2','Output3','Output4','Output5',...'Output10])
        # print(train.head())

        # summarize the data
        #for i in range(len(train_X)):
        #    print(train_X[i], train_Y[i])

        # split into samples
        #test_X, test_Y= split_sequence(test, n_steps_in, n_steps_out)
        test = series_to_supervised(test, n_steps_in, n_steps_out) #the output tyep is dataframe 
        test.reset_index(drop=True, inplace=True)

        #test = np.concatenate((test_X, test_Y), axis=1)
        #test = pd.DataFrame(test, columns = ['Input1', 'Input2','Input3','Input4','Input5','Output1','Output2','Output3','Output4','Output5','Output6','Output7','Output8','Output9','Output10'])
        # print(test.head())


        if problem ==1:
            train.to_csv('data/train/obs/train.csv')
        if problem ==2:
            train.to_csv('data/train/sg5/train.csv')
        if problem ==3:
            train.to_csv('data/train/sg7/train.csv')
        if problem ==4:
            train.to_csv('data/train/sg9/train.csv')
        if problem ==5:
            train.to_csv('data/train/ew2/train.csv')
        if problem ==6:
            train.to_csv('data/train/ew3/train.csv')
        if problem ==7:
            train.to_csv('data/train/ew4/train.csv')
        if problem ==8:
            train.to_csv('data/train/or/train.csv')

        test.to_csv('data/test/test.csv')
        print(train.columns)

        
if __name__ == "__main__": main()

finish = now.strftime("%Y/%m/%d %H:%M:%S")
print("Last edit =", finish)	

Index(['NH3_N(t-23)', 'sin_hour(t-23)', 'cos_hour(t-23)', 'sin_day(t-23)',
       'cos_day(t-23)', 'NH3_N(t-22)', 'sin_hour(t-22)', 'cos_hour(t-22)',
       'sin_day(t-22)', 'cos_day(t-22)',
       ...
       'NH3_N(t+2)', 'sin_hour(t+2)', 'cos_hour(t+2)', 'sin_day(t+2)',
       'cos_day(t+2)', 'NH3_N(t+3)', 'sin_hour(t+3)', 'cos_hour(t+3)',
       'sin_day(t+3)', 'cos_day(t+3)'],
      dtype='object', length=135)
Index(['NH3_N(t-23)', 'sin_hour(t-23)', 'cos_hour(t-23)', 'sin_day(t-23)',
       'cos_day(t-23)', 'NH3_N(t-22)', 'sin_hour(t-22)', 'cos_hour(t-22)',
       'sin_day(t-22)', 'cos_day(t-22)',
       ...
       'NH3_N(t+2)', 'sin_hour(t+2)', 'cos_hour(t+2)', 'sin_day(t+2)',
       'cos_day(t+2)', 'NH3_N(t+3)', 'sin_hour(t+3)', 'cos_hour(t+3)',
       'sin_day(t+3)', 'cos_day(t+3)'],
      dtype='object', length=135)
Index(['NH3_N(t-23)', 'sin_hour(t-23)', 'cos_hour(t-23)', 'sin_day(t-23)',
       'cos_day(t-23)', 'NH3_N(t-22)', 'sin_hour(t-22)', 'cos_hour(t-22)',
       'sin_da